# 城市经纬度匹配

In [1]:
from tqdm import tqdm #查看循环进度
import numpy as np
import pandas as pd

In [2]:
#导入经纬度数据和原始数据
p_data = pd.read_csv('E:/ShuiChanProcessed/orign_data/2015To2023ShuiChanData.csv')
LaAndLog = pd.read_excel('E:/ShuiChanProcessed/orign_data/经纬度.xlsx')

C:\Users\XuanW\AppData\Local\Temp\ipykernel_20400\3436361257.py:2: DtypeWarning: Columns (22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  p_data = pd.read_csv('2015To2023ShuiChanData.csv')


In [3]:
LaAndLog.columns

Index(['通报单位', '地级市', '经度', '纬度'], dtype='object')

In [4]:
p_data['抽检级别'].value_counts()

抽检级别
市抽    281708
省抽    254712
国抽     56759
Name: count, dtype: int64

In [5]:
# 筛选市抽的数据
new_data = p_data[p_data['抽检级别'] == '市抽']

In [6]:
#根据城市匹配经纬度数据
merged_df = pd.merge(new_data, LaAndLog[['通报单位', '经度', '纬度']], on='通报单位', how='left')

In [7]:
#查看是否有缺失经纬度的城市
filtered_df = merged_df[merged_df['经度'].isna()]
filtered_df['通报单位'].value_counts()

Series([], Name: count, dtype: int64)

In [8]:
#查看是否有生产日期缺失的数据
filtered_df = merged_df[merged_df['生产日期'].isna()]
filtered_df['通报单位'].value_counts()

通报单位
广州市场监管局    16762
郑州市场监管局     1441
深圳市场监管局     1259
沈阳市场监管局     1112
苏州市场监管局      768
           ...  
晋城市场监管局        1
南通市场监管局        1
天水市场监管局        1
鹤壁市场监管局        1
阜阳市场监管局        1
Name: count, Length: 186, dtype: int64

In [9]:
#用通报时间填补缺失的生产日期
merged_df['生产日期'] = merged_df['生产日期'].fillna(p_data['通报时间'])
#这一步不一定这样处理，怎样处理数据的缺失值可以和老师讨论

In [10]:
merged_df['生产日期']

0          2019/9/20
1          2019/9/20
2          2019/8/15
3          2019/9/21
4          2019/9/21
             ...    
281703    2023/11/25
281704    2023/11/18
281705     2023/2/15
281706    2023/11/13
281707    2023/10/15
Name: 生产日期, Length: 281708, dtype: object

In [11]:
# 保存中间数据，也可以不保存接着在一个notebook里继续处理，但最好保存一下
merged_df.to_csv('selectedShuichanWithLA.csv', index=False, encoding = 'utf_8_sig')

# 城市天气数据匹配

In [2]:
from tqdm import tqdm #查看循环进度
import numpy as np
import pandas as pd

# 导入数据
data_file = r'D:\SHUICHAN\20250405\city_match.xlsx'  # 假设文件名为 'monitoring_data.xlsx'
p_data = pd.read_excel(data_file)
data_file2 = r'D:\SHUICHAN\20250405\climate_data\2020-2023-select.xlsx'
c_data = pd.read_excel(data_file2)

In [3]:
# 转换时间格式，保证格式统一
p_data['通报时间'] = pd.to_datetime(p_data['通报时间'])
c_data['DATE'] = pd.to_datetime(c_data['DATE'])
#选取天气数据中需要的数据列
cm_data = c_data[['DATE', 'STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'DEWP', 'TEMP', 'PRCP', 'WDSP', 'MAX', 'MIN']]

In [4]:
# 在抽检数据中添加存储气候数据的列，并初始化
p_data['temp'] = np.nan
p_data['prcp'] = np.nan
p_data['dewp'] = np.nan
p_data['wdsp'] = np.nan
p_data['max'] = np.nan
p_data['min'] = np.nan

# 对于每个鸡肉生产条目
for i in tqdm(p_data.index):
    produce_date = p_data.at[i, '通报时间']
    city_longitude = p_data.at[i, '经度']
    city_latitude = p_data.at[i, '纬度']

    # 过滤相同日期的气候数据
    same_date_climate = cm_data[cm_data['DATE'] == produce_date]
    # 计算距离并筛选
    '''
    distances = (same_date_climate['LONGITUDE'] - city_longitude)**2 + (same_date_climate['LATITUDE'] - city_latitude)**2
    close_enough = distances[distances <= 32].index  # 获取符合条件的索引 
    '''
    distances = (same_date_climate['LONGITUDE'] - city_longitude)**2 + (same_date_climate['LATITUDE'] - city_latitude)**2
    close_enough = distances[distances <= 32].index  # 获取符合条件的索引

    # 计算加权温度和降水
    if not close_enough.empty:
        weights = 1 / distances[close_enough]
        temp_weighted = same_date_climate.loc[close_enough, 'TEMP'] * weights
        prcp_weighted = same_date_climate.loc[close_enough, 'PRCP'] * weights
        dewp_weighted = same_date_climate.loc[close_enough, 'DEWP'] * weights
        wdsp_weighted = same_date_climate.loc[close_enough, 'WDSP'] * weights
        max_weighted = same_date_climate.loc[close_enough, 'MAX'] * weights
        min_weighted = same_date_climate.loc[close_enough, 'MIN'] * weights

        # 存储加权平均结果
        p_data.at[i, 'temp'] = (temp_weighted.sum() / weights.sum())
        p_data.at[i, 'prcp'] = (prcp_weighted.sum() / weights.sum())
        p_data.at[i, 'dewp'] = (dewp_weighted.sum() / weights.sum())
        p_data.at[i, 'wdsp'] = (wdsp_weighted.sum() / weights.sum()) 
        p_data.at[i, 'max'] = (wdsp_weighted.sum() / weights.sum())
        p_data.at[i, 'min'] = (wdsp_weighted.sum() / weights.sum())

100%|█████████████████████████████████████████████████████████████████████████| 216276/216276 [18:13<00:00, 197.74it/s]


In [9]:
import math

p_data['temp_c'] = (p_data['temp'] - 32) / 1.8
p_data['dewp_c'] = (p_data['dewp'] - 32) / 1.8

# 定义计算相对湿度的函数
def cal_hum(row):
    dry_bulb_temp = row['temp_c']
    dew_point_temp = row['dewp_c']
    
    # 定义饱和水蒸气压力的公式
    def saturation_vapor_pressure(temp_c):
        return 6.112 * math.exp((17.67 * temp_c) / (temp_c + 243.5))
    
    # 计算干球温度和露点温度下的水蒸气压力
    e_t = saturation_vapor_pressure(dry_bulb_temp)
    e_td = saturation_vapor_pressure(dew_point_temp)
    
    # 计算相对湿度
    relative_humidity = (e_td / e_t) * 100
    return relative_humidity


p_data['relative_humidity'] = p_data.apply(cal_hum, axis=1)


p_data['max_c'] = (p_data['max'] - 32) / 1.8
p_data['min_c'] = (p_data['min'] - 32) / 1.8

In [10]:
# 统计数据中的缺失
nan_counts = p_data.isna().sum()
nan_counts

产品分类                      0
产品名称                      0
分类                        0
判定结果                      0
不合格项目分类              206442
不合格项目                206445
不合格规范列               206437
检测数值                 206638
法规限制                 207198
规格型号                   4760
商标                   131662
生产日期                      0
年                         0
月                         0
日                         0
生产企业名称                 3425
生产企业地址                 2602
生产省份                      0
抽样单位名称                  636
抽样单位地址                22935
抽样单位省份                    0
通报时间                      0
通报文号                      0
通报单位                      0
地级市                       0
通报省份                      0
抽检环节                      0
是否属于网抽                    0
经度                        0
纬度                        0
temp                      0
prcp                      0
dewp                      0
wdsp                      0
max                       0
min                 

In [11]:
output_file = r'D:\SHUICHAN\20250405\climate_match.xlsx'
p_data.to_excel(output_file, index=False)